In [91]:
import pandas as pd

tag_df = pd.read_pickle('data/tag_df.pkl')
tag_df

,name,placed_by,found_by,placed_by_id,found_by_id,placed_at_datetime,found_at_datetime,drive_distance_mi,crow_dist_mi
tag_id,,,,,,,,,
1,York,JÃ¶rn Ãorsteinn,Jon Nelson,29,26,2020-09-27 10:47:00,2020-10-02 17:53:00,NaN,NaN
2,Foster Bridge,Jon Nelson,Jeff Plotzke,26,23,2020-10-02 19:05:00,2020-10-05 17:02:00,5.98,4.80
3,Jump Course,Jeff Plotzke,Kari Alsager,23,30,2020-10-05 17:02:00,2020-10-08 18:40:00,2.96,2.18
4,Kingsley Fish,Kari Alsager,Christopher Roper,30,10,2020-10-08 18:40:00,2020-10-12 14:18:00,1.08,0.93
5,Olson Teeter,Christopher Roper,Ty,10,43,2020-10-12 14:18:00,2020-10-13 13:33:00,2.83,2.60
...,...,...,...,...,...,...,...,...,...
178,Smalley Family Cemetery,Ellen Muehlberger,David Gerdes,17,14,2023-10-28 14:57:00,2023-11-12 13:14:00,12.68,9.03
179,Huron Parkway Gallup Underpass,David Gerdes,Ellen Muehlberger,14,17,2023-11-12 13:14:00,2023-11-16 15:31:00,11.35,5.74
180,Joy/Vorheis Flamingo,Ellen Muehlberger,Dan Appledorn,17,13,2023-11-16 15:31:00,2023-11-17 11:22:00,7.44,5.55


In [133]:
tag_df.found_at_datetime - tag_df.placed_at_datetime

tag_id
1      5 days 07:06:00
2      2 days 21:57:00
3      3 days 01:38:00
4      3 days 19:38:00
5      0 days 23:15:00
            ...       
178   14 days 22:17:00
179    4 days 02:17:00
180    0 days 19:51:00
181    2 days 05:51:00
182   20 days 22:04:00
Length: 182, dtype: timedelta64[ns]

In [159]:
# a time series scatter plot with the x axis being 'placed_at_datetime' and the y axis being 'drive_distance_mi'
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime

tag_df['days_to_find'] = (pd.to_timedelta(tag_df.found_at_datetime - tag_df.placed_at_datetime).dt.total_seconds()) / (3600*24)
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=tag_df.placed_at_datetime, y=tag_df.drive_distance_mi, name="Driving Distance",
    mode='markers'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=tag_df.placed_at_datetime,y=tag_df.days_to_find , name="Days to Find",
    mode='markers'),
    secondary_y=True
)

fig.show()

In [160]:
px.scatter(tag_df, x='days_to_find', y='drive_distance_mi')

In [92]:
heatmap_df = (tag_df.groupby([tag_df.placed_at_datetime.dt.year, tag_df.placed_at_datetime.dt.month])['name']
 .count()
 .unstack( level=0)
 .fillna(0))

heatmap_df.index = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct',
                    'Nov', 'Dec']

In [131]:
import plotly.express as px
fig = px.imshow(heatmap_df.transpose(),
                labels=dict(x="Month", y="Year", color="Tags"),
                y=heatmap_df.columns,
                x=heatmap_df.index,
                color_continuous_scale='viridis',
                title='Tags Placed by Month and Year',
                )
fig.show()

TypeError: imshow() got an unexpected keyword argument 'text'

In [128]:
for index, row in enumerate(heatmap_df.transpose().values):
    for value in row:
        fig.add_annotation(x=row, y=index, text=value,
                           showarrow=False, font_size=10)

In [129]:
fig.show()

In [94]:
import plotly.graph_objects as go

import numpy as np

x0 = tag_df.drive_distance_mi
x1 = tag_df.crow_dist_mi

fig = go.Figure()
fig.add_trace(go.Histogram(x=x0,
              name='Driving Distance'))
fig.add_trace(go.Histogram(x=x1,
              name='Distance as the Crow Flies'))

# Overlay both histograms
fig.update_layout(barmode='overlay',
                  title='Distance Histograms',
                  xaxis_title='Distance (miles)')

# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

In [102]:
import json
from plotly.utils import PlotlyJSONEncoder
histJSON = json.dumps(fig, cls=PlotlyJSONEncoder)

In [103]:
histJSON

'{"data": [{"name": "Driving Distance", "x": [null, 5.98, 2.96, 1.08, 2.83, 4.32, 2.56, 3.61, 4.44, 1.43, 4.06, 1.54, 2.02, 0.5, 0.44, 0.31, 1.49, 1.68, 0.75, 0.27, 0.56, 1.14, 1.56, 5.45, 3.23, 5.26, 4.54, 1.89, 2.69, 7.24, 5.54, 3.84, 6.86, 5.33, 5.5, 2.99, 4.22, 0.8, 12.91, 11.18, 8.34, 5.02, 2.55, 5.57, 6.81, 4.12, 12.86, 5.96, 1.96, 0.64, 0.78, 6.22, 3.23, 4.86, 3.94, 4.83, 0.0, 0.22, 0.12, 0.93, 1.39, 2.39, 0.83, 1.25, 1.36, 1.13, 1.72, 1.04, 0.71, 8.06, 8.79, 2.19, 0.4, 10.1, 3.44, 1.86, 1.96, 5.3, 1.19, 14.4, 2.83, 4.32, 1.51, 6.73, 12.2, 8.38, 2.94, 2.53, 2.56, 0.83, 0.77, 4.09, 19.82, 0.43, 2.46, 8.88, 4.05, 8.86, 8.58, 0.58, 8.34, 4.32, 3.42, 4.3, 4.08, 3.31, 0.96, 5.87, 9.89, 0.33, 8.68, 7.82, 2.08, 1.61, 1.45, 1.17, 0.97, 1.7, 1.21, 1.03, 1.62, 0.95, 1.47, 8.71, 2.53, 3.64, 2.31, 5.33, 9.04, 1.72, 8.26, 14.26, 3.66, 10.26, 11.17, 8.19, 7.7, 16.29, 19.36, 0.1, 15.88, 15.82, 2.61, 0.0, 11.0, 4.39, 9.0, 5.88, 0.92, 8.21, 5.92, 2.13, 2.64, 1.33, 4.03, 1.63, 1.98, 3.12, 5.41, 1

In [100]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.embed import components
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
import networkx as nx
import plotly.graph_objects as go

person_df = pd.read_pickle('data/person_df.pkl')

G = nx.MultiDiGraph()
for index, row in tag_df.iterrows():
    G.add_edge(row.placed_by_id, row.found_by_id)
    
degrees = dict(nx.degree(G))
nx.set_node_attributes(G, name='degree', values=degrees)

edgeweights = {}
for t in G.edges:
    edgeweights[(t[0], t[1])] = edgeweights.get((t[0], t[1]), 0) + 1
    
for t in G.edges:
    G.edges[t[0], t[1], t[2]]['weight'] = edgeweights[(t[0], t[1])] * 3
    
#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Spectral8

for node in G.nodes:
    G.nodes[node]['x'] = person_df.loc[person_df.person_id==node].convex_hull_center.x.values[0] / 10000
    G.nodes[node]['y'] = person_df.loc[person_df.person_id==node].convex_hull_center.y.values[0] / 10000
    G.nodes[node]['name'] = person_df.loc[person_df.person_id==node].index[0]
    G.nodes[node]['cluster'] = person_df.loc[person_df.person_id==node].cluster.values[0]
    G.nodes[node]['cluster_color'] = color_palette[person_df.loc[person_df.person_id==node].cluster.values[0]]
    
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(G)])
nx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'cluster_color'


#Choose a title!
title = 'Bike Tag Network Graph'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("Person", "@name"),
                  ("Cluster", "@cluster"),
                  ("Degree", "@degree")]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, nx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])

#Set node sizes and colors
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=color_by_this_attribute)

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width='weight')

plot.renderers.append(network_graph)

show(plot)
#save(plot, filename=f"{title}.html")

In [98]:
save(plot, filename=f"assets/biketag_network.html")

/var/folders/v3/xvdkctpx75z05zblz25nw7l80000gn/T/ipykernel_14752/17335292.py:1: UserWarning:

save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN

/var/folders/v3/xvdkctpx75z05zblz25nw7l80000gn/T/ipykernel_14752/17335292.py:1: UserWarning:

save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'



'/Users/kate/PycharmProjects/biketag/assets/biketag_network.html'

In [101]:
components(plot)

('    <script type="text/javascript">\n        (function() {\n  const fn = function() {\n    Bokeh.safely(function() {\n      (function(root) {\n        function embed_document(root) {\n        const docs_json = \'{"3d86320f-c9a3-4285-a3bf-8a48df9da630":{"version":"3.3.2","title":"Bokeh Application","roots":[{"type":"object","name":"Figure","id":"p3188","attributes":{"x_range":{"type":"object","name":"Range1d","id":"p3186","attributes":{"start":-10.1,"end":10.1}},"y_range":{"type":"object","name":"Range1d","id":"p3187","attributes":{"start":-10.1,"end":10.1}},"x_scale":{"type":"object","name":"LinearScale","id":"p3198"},"y_scale":{"type":"object","name":"LinearScale","id":"p3199"},"title":{"type":"object","name":"Title","id":"p3191","attributes":{"text":"Bike Tag Network Graph"}},"renderers":[{"type":"object","name":"GraphRenderer","id":"p3215","attributes":{"layout_provider":{"type":"object","name":"StaticLayoutProvider","id":"p3232","attributes":{"graph_layout":{"type":"map","entries